# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

## ノック１１：データを読み込んでみよう

In [138]:
import pandas as pd
uriage_data = pd.read_csv("uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [139]:
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


## ノック１２：データの揺れを見てみよう

In [140]:
uriage_data["item_name"].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [141]:
uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

## ノック１３：データに揺れがあるまま集計しよう

In [142]:
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y/%m")
res = uriage_data.pivot_table(index="purchase_month", columns="item_name",aggfunc="size",fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
2019/02,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
2019/03,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019/04,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2019/05,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2019/06,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2019/07,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


1~2行目は日付型の定義と似付けを年月の形に変換を行っている。集計単位に合わせて日付を変更する処理はかなり行われる。

４行目は画面上に集計結果を表示している。

In [143]:
res = uriage_data.pivot_table(index="purchase_month", columns="item_name",values="item_price",aggfunc="sum",fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
2019/02,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
2019/03,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019/04,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
2019/05,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
2019/06,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
2019/07,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


## ノック１４：商品名の揺れを補正しよう

### 商品名のユニーク数確認

In [144]:
print(len(pd.unique(uriage_data.item_name)))

99


**pd.unique** でitem_nameの重複を除外したユニークなデータ件数を確認できる

In [145]:
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace("  ", "")
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data.sort_values(by=["item_name"], ascending=True)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,2019/05
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,2019/06
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,2019/06
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,2019/02
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,2019/04
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,2019/04
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,2019/01
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,2019/05


1行目の **str.upper()** で商品名の小文字を大文字に変換して統一


2,3行目の **str.repalace()** で商品名の半角・全角スペースを除去


4行目でデータ **item_name** 順にソートし画面上に表示

In [146]:
print(pd.unique(uriage_data["item_name"]))
print(len(pd.unique(uriage_data["item_name"])))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


## ノック１５：金額欠損値の補完をしよう

### 行方向に向かって欠損値があるかを調べる際には **any(axis=0)**

In [147]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

**item_price** が **True** となっているのでこの項目に欠損値があることがわかる

In [148]:
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg), "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"]==trg)] = price
uriage_data.head()

/Users/masahiro/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019/06
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019/07
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,2019/05
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019/02
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,2019/04


### １行目でitem_priceの中で欠損地のある箇所を特定する。この処理を実行することでfig_is_null変数にどの行に欠損値が存在するかが保持される。


### ２行目でループ処理を行うが、ループ条件としてlist(uriage_data.loc[fig_is_null,"item_name"].unique())というデータを用いている。

これは先ほど生成したfig_is_nullによりデータが欠損している商品名の一覧を作成する処理になっている。

まず一番大枠のlist()は変数の値をリスト形式に変換する処理になっている。

次にuriage_data.loc[fig_is_null,"item_name"]は.loc関数は条件を付与し、それを合致するデータを抽出することができる。

今回の用件とは「金額が欠損している」となるため、先ほど生成したfig_is_nullを渡すことで条件付けしている。

２番目のitem_nameは条件に合致したデータのどの列を取得するかを指定します。今回は欠損値の存在する商品名を抽出しています。

最後にunique()は抽出した商品名の重複をなくし、一意にしている。無駄なループを無くすために行なっている。

### ループ処理内の **price=uriage_data.loc[(~fig_is_null)&(uriage_data["item_name]==trg),"item_price"].max()** について

こちらはループ変数である「欠損値がある商品」を用いて、同じ商品で金額が正しく記載されている行をlocで探し、その金額を取得している。

.loc()の条件には **(~fig_is_null)&(uriage_data["item_name]==trg)** のように複数の条件を指定することが可能です。

~fig_is_nullの「~」は否定演算子といい、[fig_is_null == False]と同義

これにより、欠損値がある商品と同じ商品データから金額を取得することができる。

### 金額でデータを保管してく
### uriage_data["item_price"].loc[(fig_is_null)&(uriage_data["item_name"]==trg)]==priceでは
売上履歴のitem_price列に対して.locを行い、欠損を起こしている対象データを抽出し、先ほど生成したpriceを欠損値に代入している

In [149]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [150]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "の最大額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max()) + "の最小額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False)))

商品Aの最大額：100.0の最小額：100.0
商品Bの最大額：200.0の最小額：200.0
商品Cの最大額：300.0の最小額：300.0
商品Dの最大額：400.0の最小額：400.0
商品Eの最大額：500.0の最小額：500.0
商品Fの最大額：600.0の最小額：600.0
商品Gの最大額：700.0の最小額：700.0
商品Hの最大額：800.0の最小額：800.0
商品Iの最大額：900.0の最小額：900.0
商品Jの最大額：1000.0の最小額：1000.0
商品Kの最大額：1100.0の最小額：1100.0
商品Lの最大額：1200.0の最小額：1200.0
商品Mの最大額：1300.0の最小額：1300.0
商品Nの最大額：1400.0の最小額：1400.0
商品Oの最大額：1500.0の最小額：1500.0
商品Pの最大額：1600.0の最小額：1600.0
商品Qの最大額：1700.0の最小額：1700.0
商品Rの最大額：1800.0の最小額：1800.0
商品Sの最大額：1900.0の最小額：1900.0
商品Tの最大額：2000.0の最小額：2000.0
商品Uの最大額：2100.0の最小額：2100.0
商品Vの最大額：2200.0の最小額：2200.0
商品Wの最大額：2300.0の最小額：2300.0
商品Xの最大額：2400.0の最小額：2400.0
商品Yの最大額：2500.0の最小額：2500.0
商品Zの最大額：2600.0の最小額：2600.0


**.min(skipna=False)** の意味はNaNデータを無視するという意味

## ノック１６：顧客名の揺れを補正しよう

In [151]:
kokyaku_data["顧客名"].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [152]:
uriage_data["customer_name"].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [153]:
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("　", "")
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace(" ", "")
kokyaku_data["顧客名"].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

## ノック１７：日付の揺れを補正しよう

数値として取り込まれている「登録日」の修正

### 数値から日付に変換

In [154]:
fig_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
fig_is_serial.sum()

22

１行目のfig_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()では、顧客台帳の登録日が数値かどうかをstr.isdigit()で判定する

In [155]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[fig_is_serial,"登録日"].astype("float"),unit="D")+pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

この処理ではpd.to_timedelta()関数を用いて、数値（シリアル値）から日付に変換している。

対象データは.locを用いてfig_is_serialの条件でデータを抽出し、日付変換を実施している。

### 日付として取り込まれている対象の書式変更結果

In [158]:
fromString = pd.to_datetime(kokyaku_data.loc[~fig_is_serial,"登録日"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

### 日付更新結果

In [159]:
kokyaku_data["登録日"] = pd.concat([fromSerial,fromString])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


### 登録月の集計結果

In [160]:
kokyaku_data["登録年月"] = kokyaku_data["登録日"].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("登録年月").count()["顧客名"]
print(rslt)
print(len(kokyaku_data))

登録年月
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


### 数値項目の有無

In [161]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

0

22件だったのが0件となり、全ての数値データが日付に補正された

## ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [162]:
join_data = pd.merge(uriage_data,kokyaku_data,left_on="customer_name",right_on="顧客名",how="left")
join_data = join_data.drop("customer_name",axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,2019/06,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,2019/07,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,2019/05,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,2019/02,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,2019/04,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,2019/02,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,2019/06,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,2019/03,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,2019/07,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15,201807


left_on,right_onで結合するキーとなるデータを指定する。

left_onには引数最初に指定したuriage_dataに含まれるキー候補を記載

right_onに次に指定したkokyaku_dataに含まれるキー候補を記載

howは結合方法で「left」を指定することでuriage_dataを主としてkokyaku_dataを副として結合するという意味になる

## ノック１９：クレンジングしたデータをダンプしよう

In [163]:
dump_data = join_data[["purchase_date","purchase_month","item_name","item_price","顧客名","かな","地域","メールアドレス","登録日"]]
dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,2019/06,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,2019/07,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,2019/05,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,2019/02,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,2019/04,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,2019/02,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,2019/06,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,2019/03,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,2019/07,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15


join_dataから必要な列と任意の順番に並び替えている。カラム名を指定するだけでできるので簡単に整形できる。

In [164]:
dump_data.to_csv("dump_data.csv",index=False)

整形したデータをファイルに出力させる。

## ノック２０：データを集計しよう

In [165]:
import_data = pd.read_csv("dump_data.csv")
import_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,2019/06,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26 00:00:00
1,2019-07-13 13:05:29,2019/07,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07 00:00:00
2,2019-05-11 19:42:07,2019/05,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19 00:00:00
3,2019-02-12 23:40:45,2019/02,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22 00:00:00
4,2019-04-22 03:09:35,2019/04,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07 00:00:00


### 購入年月、商品の集計結果

In [166]:
by_Item = import_data.pivot_table(index="purchase_month",columns="item_name",aggfunc="size",fill_value=0)
by_Item

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019/02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
2019/03,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
2019/04,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
2019/05,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
2019/06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019/07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


### 購入年月、売上金額の集計結果

In [167]:
byPrice = import_data.pivot_table(index="purchase_month",columns="item_name",values="item_price",aggfunc="sum",fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
2019/02,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
2019/03,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
2019/04,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
2019/05,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
2019/06,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
2019/07,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


### 購入年月、各顧客の購入数の集計結果

In [168]:
byCustomer = import_data.pivot_table(index="purchase_month",columns="顧客名",aggfunc="size",fill_value=0)
byCustomer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019/01,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
2019/02,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
2019/03,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
2019/04,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
2019/05,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
2019/06,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
2019/07,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


### 購入年月、地域における販売数の集計結果

In [169]:
byRegion = import_data.pivot_table(index="purchase_month",columns="地域",aggfunc="size",fill_value=0)
byRegion

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
2019/01,59,55,72,34,49,57,49,42
2019/02,71,46,65,48,61,52,43,63
2019/03,64,52,57,43,52,59,51,59
2019/04,64,48,54,45,48,58,40,52
2019/05,57,52,68,48,59,65,35,43
2019/06,53,47,61,30,51,51,58,58
2019/07,76,53,61,42,54,64,47,54


### 集計期間内での離脱顧客

In [171]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on = "customer_name",right_on="顧客名",how="right")
away_data[away_data["purchase_date"].isnull()][["顧客名","メールアドレス","登録日"]]

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23


　売上履歴と顧客台帳をライトジョインし、顧客台帳を主体として結合してみる。集計期間内に購買を行なっていない顧客は「購買日」等がNaNで結合される。それを条件にデータを抽出することで、集計期間に購入を行なっていない顧客を調べることが可能。